# WeatherPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from config import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# How nmany cities in this analysis?
cnt = 6

In [2]:
# Save config information.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

# Build partial query URL
query_url = f"{url}appid={api_key}&units={units}&q="
query_url

'http://api.openweathermap.org/data/2.5/weather?appid=f0c215066f97fe3149c5735af95697a4&units=imperial&q='

## Generate Cities List

In [3]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

print(lat_lngs)
print(cities)


[]
[]


In [4]:
# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=cnt)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=cnt)
lat_lngs = zip(lats, lngs)

In [5]:
lats

array([ 62.39806902, -13.73273023,   5.93883057,  11.58194748,
       -78.2509466 , -87.61847641])

In [6]:
lngs

array([ 142.40898179,  -72.40386552, -155.65013741,   86.62989837,
        142.44708282,   40.38710489])

In [7]:
print(lat_lngs)

In [8]:
lat_and_lng = list(zip(lats,lngs))
lat_and_lng


[(62.39806902138818, 142.40898178718686),
 (-13.732730231114516, -72.40386552123903),
 (5.938830572899363, -155.65013741474553),
 (11.581947475313356, 86.62989837037708),
 (-78.25094660036365, 142.44708282071923),
 (-87.61847640511606, 40.387104893941995)]

In [9]:
# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    print(city)
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
        print(cities)



ust-nera
['ust-nera']
anta
['ust-nera', 'anta']
hilo
['ust-nera', 'anta', 'hilo']
port blair
['ust-nera', 'anta', 'hilo', 'port blair']
hobart
['ust-nera', 'anta', 'hilo', 'port blair', 'hobart']
port alfred
['ust-nera', 'anta', 'hilo', 'port blair', 'hobart', 'port alfred']


In [10]:
# Print the city count to confirm sufficient count
len(cities)

6

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [11]:

# set up lists to hold reponse info
lats2 = []
temps = []
humids = []
clouds = []
winds = []
hems = []
orig_lats = []



count = 0

# Loop through the list of cities and perform a request for data on each
for city in cities:
    print(f"{count}: {city}")
    print(lat_and_lng[count])
    orig_lats.append(lats[count])
    print(f"orig latitude is: {lats[count]}")

    count = count +1
    print(count)
    response = requests.get(query_url + city).json()
    print(response)
    

    lat2 = (response['coord']['lat'])
    temp = (response['main']['temp'])
    humid = (response['main']['humidity'])
    cloud = (response['clouds']['all'])
    wind = (response['wind']['speed'])
    print(f"{count}: {city}")
    hem = "Nothern" if (lat2 > 0) else "Southern"
    print(f"located in {hem} hemisphere")    
    print(f"latitude is: {lat2}")

    print(f"temperature: {temp}")
    print(f"humidity is: {humid}")
    print(f"cloudiness is: {cloud}")
    print(f"wind speed is: {wind}")
    

    lats2.append(response['coord']['lat'])
    temps.append(response['main']['temp'])
    humids.append(response['main']['humidity'])
    clouds.append(response['clouds']['all'])
    winds.append(response['wind']['speed'])
    hems.append(hem)

        
    print()

0: ust-nera
(62.39806902138818, 142.40898178718686)
orig latitude is: 62.39806902138818
1
{'coord': {'lon': 143.2, 'lat': 64.57}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 77.86, 'feels_like': 78.33, 'temp_min': 77.86, 'temp_max': 77.86, 'pressure': 1012, 'humidity': 47, 'sea_level': 1012, 'grnd_level': 958}, 'visibility': 10000, 'wind': {'speed': 2.51, 'deg': 19}, 'clouds': {'all': 95}, 'dt': 1595540433, 'sys': {'country': 'RU', 'sunrise': 1595523769, 'sunset': 1595592256}, 'timezone': 36000, 'id': 2120048, 'name': 'Ust-Nera', 'cod': 200}
1: ust-nera
located in Nothern hemisphere
latitude is: 64.57
temperature: 77.86
humidity is: 47
cloudiness is: 95
wind speed is: 2.51

1: anta
(-13.732730231114516, -72.40386552123903)
orig latitude is: -13.732730231114516
2
{'coord': {'lon': 76.3, 'lat': 25.15}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'base': 'station

In [12]:
# create a data frame from cities, lat, and temp
weather_dict = {
    "city": cities,
    "hemisphere": hems,
    "lat": lats2,
    "orig lat": orig_lats,
    "temp": temps,
    "humidity": humids,
    "cloudiness": cloud,
    "wind speed": winds
}
weather_data = pd.DataFrame(weather_dict)
weather_data

,city,hemisphere,lat,orig lat,temp,humidity,cloudiness,wind speed
0,ust-nera,Nothern,64.57,62.398069,77.86,47,0,2.51
1,anta,Nothern,25.15,-13.732730,81.81,69,0,1.92
2,hilo,Nothern,19.73,5.938831,78.06,61,0,3.36
3,port blair,Nothern,11.67,11.581947,81.50,74,0,3.60
4,hobart,Southern,-42.88,-78.250947,37.47,80,0,10.29
5,port alfred,Southern,-33.59,-87.618476,63.55,41,0,10.16


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


In [ ]:
# Extract relevant fields from the data frame


# Export the City_Data into a csv


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots

In [ ]:
# Create Northern and Southern Hemisphere DataFrames

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression